In [ ]:
from google import genai
from google.genai import types
import random

with open("api_key.txt", "r") as f:
    api_key = f.read().strip()

client = genai.Client(api_key=api_key)

topics = ["technology", "science", "history", "art", "literature", "politics", "economics", "philosophy"]

sys_instruction="All prompts should be answered with an in depth paper with an introduction, middle and end structured into chapters that is about 4 pages, written in markdown and include sources. Dont answer anything with less than 4 pages. Dont write anything other than the paper."


for _ in range(30):
    topic = random.choice(topics)
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruction),
    contents=[f"Write a paper about {topic}. The specific topic is up to you."],
    )

    try: 
        with open(f"outputs/{_}_{topic}.md", "a") as f:
            f.write(response.text)
    except:
        pass

